In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta

def plot_dividends(ticker):
    try:
        stock = yf.Ticker(ticker)
        dividends = stock.dividends

        if dividends.empty:
            raise ValueError("No dividend data available for this stock.")

        dividends.index = dividends.index.tz_localize(None)

        cutoff_date = datetime.now() - timedelta(days=365*10)
        div_last10 = dividends[dividends.index >= cutoff_date]

        freq = pd.infer_freq(div_last10.index)
        is_quarterly = 'Q' in str(freq) or len(div_last10) >= 40

        plt.figure(figsize=(14, 7))

        if is_quarterly:
            ax = div_last10.plot(kind='bar', color='teal', alpha=0.7)
            plt.title(f'Quarterly Dividends for {ticker} (Past 10 Years)', pad=20)
            plt.xlabel('Dividend Date', labelpad=10)
            plt.ylabel('Dividend per Share ($)', labelpad=10)

            for p in ax.patches:
                ax.annotate(f"{p.get_height():.2f}",
                            (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha='center', va='center',
                            xytext=(0, 10),
                            textcoords='offset points',
                            fontsize=9)

            plt.xticks(rotation=45, ha='right')
            ax.xaxis.set_tick_params(labelsize=9)

        else:
            yearly = div_last10.resample('Y').sum()
            yearly.plot(kind='line', marker='o', markersize=8, linewidth=2)
            plt.title(f'Annual Dividends for {ticker} (Past 10 Years)', pad=20)
            plt.xlabel('Year', labelpad=10)
            plt.ylabel('Dividend per Share ($)', labelpad=10)

            for x, y in zip(yearly.index, yearly.values):
                plt.annotate(f"{y:.2f}",
                             (x, y),
                             textcoords="offset points",
                             xytext=(0, 10),
                             ha='center',
                             fontsize=9)

        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"❌ Error: {e}")

ticker_input = widgets.Text(
    placeholder='Enter stock ticker (e.g., AAPL, KO)',
    description='Stock Ticker:',
    disabled=False
)

compute_button = widgets.Button(
    description='Show Dividends',
    button_style='info'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        ticker = ticker_input.value.strip().upper()
        if ticker:
            print(f"Analyzing {ticker} dividend pattern...")
            plot_dividends(ticker)
        else:
            print("Please enter a valid stock ticker.")

compute_button.on_click(on_button_click)
display(widgets.VBox([ticker_input, compute_button, output]))